<h1>1. Import Library </h1>

In [1]:
import os
import numpy as np
import transformers

import tensorflow as tf
from functools import partial

import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset

from transformers import BertTokenizerFast
from transformers import BertConfig,  TFBertForQuestionAnswering
from transformers import DistilBertTokenizerFast
from transformers import DistilBertConfig, TFDistilBertForQuestionAnswering

2022-08-29 06:16:44.687934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 06:16:44.689110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 06:16:44.689800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-29 06:16:44.691833: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

<h1>2. Load Dataset

In [2]:
dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


**Data Structure**

In [4]:
for i in range(1):
    print(dataset['train']['id'][i]) 
    print(dataset['train']['title'][i])
    print(dataset['train']['context'][i])
    print(dataset['train']['question'][i]) 
    print(dataset['train']['answers'][i])

5733be284776f41900661182
University_of_Notre_Dame
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


<h1>3 Data Preprocessing

<h2>a. Calculate end Index of data

In [5]:
def end_index_calculation(answers, contexts):
    
    corrected_answers = []
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'][0]
        answer['answer_start'] = answer['answer_start'][0]
        
        end_idx = answer['answer_start'] + len(answer['text'])        
        answer['answer_end'] = end_idx
        
        corrected_answers.append(answer)
    
    return corrected_answers, contexts

In [6]:
train_answers, train_contexts = end_index_calculation(dataset["train"]["answers"], dataset["train"]["context"])

In [7]:
test_answers, test_contexts = end_index_calculation(dataset["validation"]["answers"], dataset["validation"]["context"])

In [8]:
train_questions = dataset["train"]["question"]
test_questions = dataset["validation"]["question"]

<h2>b. Tokenization</h2>

**Import tokenizer module**

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

**Tokenize train data**

In [10]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True, return_tensors='tf')

**Tokenize test data**

In [11]:
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True, return_tensors='tf')

<h2>c. View Tokenize data<h2>

In [12]:
train_questions[0]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [13]:
test_token = tokenizer(train_contexts[0], train_questions[0], return_tensors='tf')

In [14]:
test_token['input_ids'].numpy()

array([[  101,  6549,  2135,  1010,  1996,  2082,  2038,  1037,  3234,
         2839,  1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,
         8514,  2003,  1037,  3585,  6231,  1997,  1996,  6261,  2984,
         1012,  3202,  1999,  2392,  1997,  1996,  2364,  2311,  1998,
         5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,  4828,
         2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,
         2310,  3490,  2618,  4748,  2033, 18168,  5267,  1000,  1012,
         2279,  2000,  1996,  2364,  2311,  2003,  1996, 13546,  1997,
         1996,  6730,  2540,  1012,  3202,  2369,  1996, 13546,  2003,
         1996, 24665, 23052,  1010,  1037, 14042,  2173,  1997,  7083,
         1998,  9185,  1012,  2009,  2003,  1037, 15059,  1997,  1996,
        24665, 23052,  2012, 10223, 26371,  1010,  2605,  2073,  1996,
         6261,  2984, 22353,  2135,  2596,  2000,  3002, 16595,  9648,
         4674,  2061, 12083,  9711,  2271,  1999,  8517,  1012,  2012,
      

In [15]:
' '.join(tokenizer.convert_ids_to_tokens(test_token['input_ids'].numpy()[0]))

'[CLS] architectural ##ly , the school has a catholic character . atop the main building \' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms up ##rai ##sed with the legend " ve ##ni ##te ad me om ##nes " . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the gr ##otto , a marian place of prayer and reflection . it is a replica of the gr ##otto at lou ##rdes , france where the virgin mary reputed ##ly appeared to saint bern ##ade ##tte so ##ub ##iro ##us in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary . [SEP] to whom did the virgin mary allegedly appear in 1858 in lou ##rdes france ? [SEP]'

<h2>d. Mark start and end encoding</h2>

In [16]:
start_positions = []
end_positions = []

for i in range(len(train_answers)):
    start = train_encodings.char_to_token( i, train_answers[i]['answer_start'])
    end = train_encodings.char_to_token( i, train_answers[i]['answer_end'] - 1)
    
    if start == None:
        start_positions.append(tokenizer.model_max_length -1)
    else:
        start_positions.append(start)
        
    if end == None:
        end_positions.append(tokenizer.model_max_length -1)
    else:
        end_positions.append(end)
        
train_encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [17]:
start_positions = []
end_positions = []

for i in range(len(test_answers)):
    start = test_encodings.char_to_token( i, test_answers[i]['answer_start'])
    end = test_encodings.char_to_token( i, test_answers[i]['answer_end'] - 1)
    
    if start == None:
        start_positions.append(tokenizer.model_max_length -1)
    else:
        start_positions.append(start)
        
    if end == None:
        end_positions.append(tokenizer.model_max_length -1)
    else:
        end_positions.append(end)
        
test_encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

<h1>4. Create data generator</h1>

In [18]:
train_batch_size = 4
test_batch_size = 8

In [19]:
def data_gen(input_ids, attention_mask, start_positions, end_positions):
    """ Generator for data """
    for inps, attn, start_pos, end_pos in zip(input_ids, attention_mask, start_positions, end_positions):
        
        yield (inps, attn), (start_pos, end_pos)

In [20]:
train_data_gen = partial(data_gen,
    input_ids=train_encodings['input_ids'], attention_mask=train_encodings['attention_mask'],
    start_positions=train_encodings['start_positions'], end_positions=train_encodings['end_positions'])

train_dataset = tf.data.Dataset.from_generator(train_data_gen, output_types=(('int32', 'int32'), ('int32', 'int32')))

In [21]:
train_dataset = train_dataset.shuffle(1000)

valid_dataset = train_dataset.take(10000)
valid_dataset = valid_dataset.batch(train_batch_size)


train_dataset = train_dataset.skip(10000)
train_dataset = train_dataset.batch(train_batch_size)

In [22]:
test_data_gen = partial(data_gen,
    input_ids=test_encodings['input_ids'], attention_mask=test_encodings['attention_mask'],
    start_positions=test_encodings['start_positions'], end_positions=test_encodings['end_positions'])


test_dataset = tf.data.Dataset.from_generator(test_data_gen, output_types=(('int32', 'int32'), ('int32', 'int32')))

test_dataset = test_dataset.batch(test_batch_size)

<h1>5. Declared model</h1>

In [23]:
config = BertConfig.from_pretrained("bert-base-uncased", return_dict=False)

pretrained_model =  TFBertForQuestionAnswering.from_pretrained("bert-base-uncased", config=config)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
input_ids = tf.keras.layers.Input([None,], dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input([None,], dtype=tf.int32, name="attention_mask")

out = pretrained_model([input_ids, attention_mask])

model = tf.keras.models.Model([input_ids, attention_mask], outputs=(out.start_logits, out.end_logits))

In [25]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc = tf.keras.metrics.SparseCategoricalAccuracy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [26]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

<h1>6. Model fitting</h1>

In [27]:
model.fit(train_dataset, validation_data=valid_dataset,epochs=3)

Epoch 1/3


2022-08-29 06:18:56.070108: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


19400/19400 [==============================] - 6480s 333ms/step - loss: 2.7412 - tf_bert_for_question_answering_loss: 1.4362 - tf_bert_for_question_answering_1_loss: 1.3050 - tf_bert_for_question_answering_sparse_categorical_accuracy: 0.5927 - tf_bert_for_question_answering_1_sparse_categorical_accuracy: 0.6332 - val_loss: 2.1333 - val_tf_bert_for_question_answering_loss: 1.0785 - val_tf_bert_for_question_answering_1_loss: 1.0548 - val_tf_bert_for_question_answering_sparse_categorical_accuracy: 0.7098 - val_tf_bert_for_question_answering_1_sparse_categorical_accuracy: 0.7326
Epoch 2/3
19400/19400 [==============================] - 6473s 333ms/step - loss: 1.6819 - tf_bert_for_question_answering_loss: 0.8974 - tf_bert_for_question_answering_1_loss: 0.7845 - tf_bert_for_question_answering_sparse_categorical_accuracy: 0.7186 - tf_bert_for_question_answering_1_sparse_categorical_accuracy: 0.7619 - val_loss: 2.1092 - val_tf_bert_for_question_answering_loss: 1.0664 - val_tf_bert_for_question

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
tf_bert_for_question_answering  TFQuestionAnsweringM 108893186   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
Total params: 108,893,186
Trainable params: 108,893,186
Non-trainable params: 0
__________________________________________________________________________________________________


<h1>7 Save Model</h1>

In [29]:
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('tokenizers'):
    os.makedirs('tokenizers')

In [30]:
model.get_layer("tf_bert_for_question_answering").save_pretrained(os.path.join('models', 'bert_model'))
tokenizer.save_pretrained(os.path.join('tokenizers', 'bert_tokenizer'))

('tokenizers/bert_tokenizer/tokenizer_config.json',
 'tokenizers/bert_tokenizer/special_tokens_map.json',
 'tokenizers/bert_tokenizer/vocab.txt',
 'tokenizers/bert_tokenizer/added_tokens.json',
 'tokenizers/bert_tokenizer/tokenizer.json')

<h1>8. Model Testing</h1>

In [31]:
model.evaluate(test_dataset)

1322/1322 [==============================] - 280s 211ms/step - loss: 2.3922 - tf_bert_for_question_answering_loss: 1.2145 - tf_bert_for_question_answering_1_loss: 1.1778 - tf_bert_for_question_answering_sparse_categorical_accuracy: 0.6936 - tf_bert_for_question_answering_1_sparse_categorical_accuracy: 0.7222


[2.3922159671783447,
 1.2144622802734375,
 1.1777504682540894,
 0.6935666799545288,
 0.7222327589988708]